In [11]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [12]:
# Importing the dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [13]:
# Preparing the training set and the test set
training_set = pd.read_csv('training_set.csv')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('test_set.csv')
test_set = np.array(test_set, dtype = 'int')

In [14]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [15]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [16]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [18]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.347552208116566
epoch: 2 loss: 1.0100239018306674
epoch: 3 loss: 0.9899552653401554
epoch: 4 loss: 0.983409733168408
epoch: 5 loss: 0.9801102859794218
epoch: 6 loss: 0.9783306696283256
epoch: 7 loss: 0.9772230760206483
epoch: 8 loss: 0.9765539533840917
epoch: 9 loss: 0.9759521024773594
epoch: 10 loss: 0.9755646719100741
epoch: 11 loss: 0.9752691750719578
epoch: 12 loss: 0.9750754809559671
epoch: 13 loss: 0.9747863156013553
epoch: 14 loss: 0.974521692563073
epoch: 15 loss: 0.9743868812897604
epoch: 16 loss: 0.9743234041433538
epoch: 17 loss: 0.9742060630349182
epoch: 18 loss: 0.9740831721721069
epoch: 19 loss: 0.9741085321901034
epoch: 20 loss: 0.9738770455462931
epoch: 21 loss: 0.9737450500542527
epoch: 22 loss: 0.9736409504787895
epoch: 23 loss: 0.9734747169499968
epoch: 24 loss: 0.9733123110640378
epoch: 25 loss: 0.9726667238259846
epoch: 26 loss: 0.9720728098820459
epoch: 27 loss: 0.9713710876281774
epoch: 28 loss: 0.9704779337068687
epoch: 29 loss: 0.97049030955657

In [22]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9212857495993412
